# Deep Image Prior — Colab Playground

This notebook lets you **run the Deep Image Prior (DIP)** experiments end‑to‑end on Google Colab:

- Denoising, Super‑resolution, Inpainting
- Architecture/depth ablations
- Optimizer & learning‑rate sweeps
- Sampling / ensembling (multi‑seeds)
- LR → smoothness analysis (TV & FFT)

> DIP: Ulyanov, Vedaldi, Lempitsky (CVPR 2018). This Colab uses a small, self‑contained research codebase.


## 0) Environment check

In [ ]:

import torch, sys
print("Python:", sys.version)
print("Torch:", torch.__version__, "CUDA:", torch.cuda.is_available())
!nvidia-smi -L || echo "No GPU visible (OK but slower)."


## 1) Install lightweight dependencies

In [ ]:

%%bash
pip -q install --upgrade pillow tqdm matplotlib scipy


## 2) Unpack the (embedded) DIP research code

In [ ]:

import base64, os, zipfile, io, glob
os.makedirs("/content/dip_research", exist_ok=True)
data = base64.b64decode("""UEsDBBQAAAAIAIhQaFsfdoJeJAAAACMAAAAQAAAAcmVxdWlyZW1lbnRzLnR4dMsrzS2o5ArIzMnJL+cqKUzJ5cpNLCnIyS/JyUziKk7OBEoCAFBLAwQUAAAACACIUGhbryEgECoAAAAoAAAACQAAAFJFQURNRS5tZFNWcElNLVDwzE1MT1UIKMrML1IoSi1OTSxKzlDIzixR0HDOz0lM0uQCAFBLAwQUAAAACACIUGhbg8INAegFAAApEQAACAAAAHRyYWluLnB5lVdbi+M2FH7Pr9BLsb3rcSfTdqEZVOjDvpaFlr4MQWhs2dHGllxJzkyy7H/v0cW3xA5tIKDLuV++I29400plEFVVS5VmKZI6RV+1FCkyUuWHTalkgwreZo0sWK1RYGjokRF3NFIYqo86y2XTSDGjsxcjWWd4rTPNWNET2TVhJ6bO5sBFdU3KG1oxwmVPXktaEHeYIk1PzK+vuRpmFM8Hg1stFJBr3hD4tzWzDhpakxNVnBouBam3KTrw6kBKxf4hTDBVna+lMqrqszay7eV+tgd/wkHL1EgsW8ObrGKGuBW/MNUzzA6nHExRCDkYbsM489WebDYFK5FLUpzsNgh+tMV93rLfVdU1TJgvdqfiJBBktCgIDXdx9PBgUxGlCBzsuGIF/kt1EIn8IHnONH6JCiYk1yxKI92BRYppWHLRUi5MtF8V6zJwLTd5XiKtDNCBL7SrDf5DCrYqtPG2zmgXReqc1la7ObcMg6Ejz8+rwitFz8BCc5t6HEFKFRQqWB2tsgg2NT3q/H6MnT+ImMgLlsMCgheOcilOgr3fi2DBWnNY9OGXZadLRk3n8rPA82ndcX3k7Yrji3q4IPmyYT89rYfKFhHRpugZSyjkCetjtl3WdiFfuTFMrfI9ruocmmqaJFrQZpYkXYFN4ThSjW6VhHBE9WtZ6Tvpqdcs2rKHp2VXGmlXXbPqyq+r2t4YwJAhUEauSP9vJLixWALpXS6Op8fHdV4HcMQi3CLvOmPBThDiaezzrqAR4qWfJJndZlwTeqK8pq814BiCicJQlLfdetfZ4bDc2ytxdyOh4LMyUJ3QP7L3dl1NLSs/gpZ7MPitGPScsPwOdq0EDWDrsLkBjBygGc5xAOtnmKmZnYNgk47tTdabmCL2zjXMg6MHTMd7NQ0DBxyCJB9k7APqN/7er70A+arxOCP9fRiXFvWwO7CrJDMynnJW5oaxMve4bHYDmc+lBWknCtBOUfztu9v0VHb6YDyOlp0ng650CL73Stxmia+fQz7GUwI7KhDXTv0OKQrog/6mdcc+KyVVHH1R8sQLhvxQQaWEceyFQYSzUBWD2S+RGz17PEzfeNACbwjAGAwxzvSBtuzl4Wm3vwmkM3h4+8SDEyn68MGp8GTuAYXHt5QnhJmRIge7Ph5uCS+zztijUfN2bwsUxgYOJQDLFPVzwR/2OzAbcD/EF1Y3Fl9CUSkqChHDS2imeuLtfrb1NriiDNI+eB/6ARCEX7K8pk0bP2zTbajS1uDZWyh2IbCNReHtxqBXkhQJWHurB7oU1cof1bCeQqU/nZ6kqEdhf9fvQqf5ZxuevuGcFRgK4B2AoAUUhXEeLBihcaz78ey3x6sOOEBn45e9W7uCM5BUqE0x9OQA08lY0T4IUCEWTJ5tlDLwWRLovCIe6/QCojC6oI9okjZS8yOLLyEF/SgdbO0Peksvg7Qz9uUHUkFnLbXGtlqzghpK7DY+u6wnN403ZAXa08/Q0RX7s8CYgwSowTiZX4UimLr3jM5Pc1Nu7HjyhsBV9krz4xtVji8Acz3T4ByxKrRhbRzMmPQ6BGF3wzATOzCPXG/cHELQRW/3XMoZ0JvWvR/J7E6x1rsEC3jfx57W+ZRa+K2Mc6Z9iWx1AARx83F7LeElMie4ci+C+PYrJghNbIE1cdILPJThm2Zgvf7YuWac6bXlnFFoEVHEIG9+CeUQW0uTH3xj9sMU40fkSh/jq5J/2N5Ww/gxN8ZFAyCYQ/ZVQj9czc7SxYh8s4p3j5+K71krqii5sSy0OYIW6dc+qS4s9sMwAhTThhqGvV7AUkPzQ5xk8DSxMDTY7fKxUMivitGjb/X7H6+jj4hqROzWsblV3NfMXcejkguwceKsq0n4ghTxPTabQanOmf28j5I0eosSa0O5cx/8WdEBPluatEy5gA9BeCkmmzsxfGXwdHFhs6NXSOPH7xAb79It9X3nLN11Iv+bf06DzVRm4FNr5mCZvSm4AWPU3CBLDoo24CIhdtgQAkhGiH3MEQIPFP+q2/wLUEsDBBQAAAAIAIhQaFuqtOB3ygIAACYGAAATAAAAZXhwZXJpbWVudF9zd2VlcC5weXVUTW+cMBC976+w9mJoibOtkh424tBDr1VUVb1EK+TAwDqA7dpmk02U/94ZYJddmkVC4PF7M8+ej4VqrXGBSVdZ6Twk7MkbnTAVwAVjGp8wg6/vHq0zOXj63/tFASXr8VG8XjB8pE0PLsR3V3Ut6HBPKxfFI0DIosjkuBfxq6sgfc0T5uBvpxwU6W/XYfx8axQGSh94AdooDzzhvrPgHHj8VdpKpQPfXHSrWlnB3G989xG0CojDs8iuCelPo+Gi03bQeob90KXPZUPRw95CikInzs1F55VTxYHig5soPDe6VJW/JkTmQbp8KyhB/EJwuYOsUO5EKXed9tf+GcDyQYCD0DlN9D5d5MBjkiilLV7tMaPpmOB+9azClhkLOpKCxCTc8ZhJz8o1o3VKqkRjZBGVA6OGvU891hYUESEEGaIYhe9k01GCyfpQb1hpHKuZ0j1l05ONF62sAY/iMeDhWAmDF+VDZuohpz2U2LlpHw15OJatwHItujxEn4Zo46HoycsqfavXuyFusiPeq7IRRU96T/H7HQuySpfZUjwZvJJy+Va/X73t3penJHQkMGBLpzp6N11IUb2VYTtwT+Wj0wmZt0X6gM0k4AXyLsjHBhIeHKZA2D1W+qFBpKBvMlW2FP1PcpZwjJscXfcPbmvAAseKikhqBVgjvYV39Ilj8lCADdsZaLTdDogSJBYMNd8ZaDJ/u0Hgf7GNDapVr+BmvBM7l4VsRx3NHEeGlVh9GbYpsTIoo+cyTje+rlarON4cpaiSYbWGNV31ZxwoQ8OTaYahC07TacycMMZ+ppiYSVrMA4zkw2Ban90EIrTB8SpogKwZZtcD+0Ml+cM54yJ+78xOFcCGEdOX1+iJjU17N4lpx4Kg76TCOkRHy1+d1kpX62XClmwsXGRSx03jW+A4ICvNWcgPfYQqs0zLFrIMT5JlNAeyDK9hGAiLf1BLAwQUAAAACACIUGhbLcNzq+kDAAAECgAAEgAAAHNhbXBsZV9lbnNlbWJsZS5weXVWy27jNhTd+yu4k5wqrJxOu0jARRfdFgXazSAwBEa8ljkWSQ1JZcYZzL/38iFLTixvzMd9HF6ee6iNVIOxnnDbDdw6qIhxFfHGtseKfHFGbw7WKOK/CkWyaRinVSEHOnrZO+oAxLQfxg28gj37o9Tde1OpeAeNNJN5b7ho4mJFHH+FNJ69lBHQu8la8RM0cWm28NydHG2NUkZf2YWN2cwMXiragW/iSL6BnayvFpceYDnWwlEVUlwhDiubjYADiXUrt48bgj8+sKmU9E/bjQq0/yfMbLnNBpQL0fC8Vxb39wFmURELX0dpQbD/7Ii1aI9GtuDYcyFAG+mgqAo3IiILDodSD1xqX+xXw8Y6vo+7fbpl2nm0w7Pwsffsb6NhNahKWK9sb4Z0Le9Ddn8egCHQ2efTanANSxzFGOa3owsY/PFm9N9vOxyA+zFW7obPH+uQpG7a24l+e1h1ulBpeRouuFo5TW+nDAfk1iJHTevdOjQf2CmNvn2o3UNdr1wNNuhtn/U68K6z0HEPyyMp4LpYMjUt4J+QOFinpoBX9FiGakfBCyIPSXpomFLpGv7KZc9feuwvgioApGiHsViNG/VDyKu621G7X0E7UBgmXIAFZIIO7rFNQwCHzRl72VtsqgZJXYYiVUEWUQ7NS9BEZH4IG5DnZn+ndNEnYYsKxWaxKkMginyuSGQUi/M4xPCjD0uYhbojH+B5tw+JkOHJLA4rMlE4LU6zLfWmzKhi6jeWSmi5FrrcVYtMc4b7h/1yhglTCJYj3dV0F6MhldmVOpbxPKF0XAEyEEtXEY3jBOtiV5HepqXeJmAHg3rr8fwEoXWQSjKTONc0J6UYwjSd5aLEKzuzVMU3HPfGORYugwrueROm5TkcJu/RF96evnEbHUMk52HI4vtN+mNmmJ6iP06EmFJgvT1vj+WWtj1XQ1lXu8wOheS4CD3Luo9JwvtwQs5jMTidOFgR+C4dVu40CW8uccqfJmi/vDs8BZvfQ9yM/x+uGL4jT9mPn3GCPcPjE8jY/D48JpvnIsnwniGuMPrgMT0jF4co8Ht2eeMQhIrMd3irbMmgx/0HYDHo5ektU5aK3N3F4PmMo0ep2F8o4QIjwjdFmWkRPybcNrSAa1nx16V1Z4Kc2dyou7quf3EVDxSoYr6M6CmmonwYQIvyjPP58wIZE65t4P5Ivxi81eW9HYrY2D/cY/0gftJBd8U2Yf+cL88hQU5lCI8opWJ1VqSuY59pEMGy3qYyX3STZblMMhasgkaiHX3l/QguCcoMEB3XIRaTnmVwT2SwMkjgv2ghyPXuBoE0TWjRpkEUTRNo3DR45YnPm/8BUEsDBBQAAAAIAIhQaFuSG1/JqQUAAI4PAAASAAAAYW5hbHl6ZV9scl9iaWFzLnB5pVfdbts2FL7PU/COcqpocbcOqAMO2M3Qi6EotmI3hkEwFiWxESWOpJy4QYG+wl5xT7JzSP0mdreuvrDJw4/n/4e+UNq01hNhSyOskylpXUo+uLZJiW/tvkqJFt7Ura/VbWaOuCLCEVP7i8K2muTKZJ1XtcuUFqXkqiU9y7oVOQ/ElDhxkHE93dJtLms3oLW4kzyQJoQX7s5l+1brtlng8GCCtcYrnZXS87BSH6Ud0Avi/Ia0AsxzmUYRC431gnc0TUtv1X7EGddYdI8XNT8Iq4RXbcPrdUoqVVa8sPJPLhtpy+NFLgsSHJusNhcEPsKwwdfZz7bstGz8O9zZZNUDMpHnXPRnCb26QnNpSoBtp6zM2XvbgU/3Vav20rEtzWXTKidpSl0HllnpYKkaI1Tj6e4s2xCPp3xXN6egpQcc2CK62rO3bSPPMtVR1wX2JEu3FzVK90cjGSg63fnhLPNGzvWgHe5Pc8+l8dVJ7q9OXyik8F3w3Ik7P55XSTV8f1rQ9y9PSxrzcW6KK3M6D2nYU5ELDT9WO2Nb84VQ1hYVb4DkGH0xqFNAQvv/FF/lsSAgi0/bv76+vj4rO5cHUHpuzL7LBSWqiB0kw22mHBcHoWpxW0MtEKh8SejedGfiFxpGrhY+sl3jvqstv1XC0aiOlRC0BhmEikIWDuoIqw4Sj8ONpIZSRTJ0tlv4wlpCnqhzSso+Q2NthvbDpk6U4L0MkiwlIcws7MMSuHUeScA0c5UwcrveIV9IuwgLy5QMeRWJw26V+TaJSkRDPrLoKyuavEnW6UzSJOHq5W6+A4GRBes5XV5n68ANUowtWl8S7EEnCehl0oKTMF20jGqNuJTUlqHHdIuR6DS7zl5HDSvlQMUj2+5uyK10nmMXZFdr+brfK10GZwZ00UIP9uA2AhaVMnpyyrLe4b2uGUhueWlFnkA2HFl0/kdY161zDEOW5cILjtvkiD7oz7Jbsb+7FzZcRE7OS9M3UvzcK1/1WdgMEibR+DkeRnHZvhbaJNfperWAWGnYI0Xl6Ub5F+uUomi6CeWVBDXgUCerVUr9YaA/Hw/J8bCakFUxIJ8OjSUO3czB4gGNe0CEdB5hnxYKQ+WV4HtHGhjWGJKlyb1N28gaWvuOPWFd+pHzs5vAfHn5pzEbNrPEWGJmKXI8QGl4sa8ScLjpkqWEPssyYYxs8gS4LAq9P05HdqHUNQy6cb6yftxCPuB4v5PQRVwisqGjpEQ+ABfe3sV+GG71dRQzJW7gSl9WN+hqNj1n4CT8zov4BlvJAoI+nM7RbUiLjW8sE/ngrWCPMXwBgsnO2DTINxEDEyHMyx0DS3D17MYw78cLYRLv2PioAZ10aH4OKp3Nu8pm96wfBabjWyuJUlJyeRmYR4zrtIb0xnm1G4u+tlj0IoOJNGUdhi3GjLB5YxZYymlg3Xdk8CI4blm/8FprEgimEb7KPrQQ63k0Cwoj4bG2nzJ8tVKomHu6widqsQnv2CzvoKiDBkWqGngrefZykgAuDHrNq2X2XE3wEB/FZ6WHTBxUME1JZzUDKcxQMvbqm8lfQ3Y/gup0A36ghWqgWUylDqfbqfJ3AwC7SziCxUjERhKIsADiWIJYeBstHpJthY/jZKzG9Gq9CrGqMFSo3271KSoNUWNbt0W9dgHiEDIqDkb4Q0SMCp3BVcUch6qdxpk5bLL4FDhoCH88skKVMESxxHGHf0kgmSCPDi4F6J20jLa0P30I1ZJAyy5HEjxCoOGjkT3l2FPe/0GSX9ergeyVx6tAPTgyoTH6oMLZlITo8IODnIjpAK8Co9j6Vf+C+oIB4LNvNODNLyTOkCcmvIER8/fnv3DI9ICvNKkq/p9J5lstevf7299IgrPuiUl4EOhfaUnIsfO2/HvHoTEjj1/sN2PWLpoONBvO8dnFOXRsznFwcQ4tO06wi38AUEsDBBQAAAAIAIhQaFvjfwBwCgEAAJICAAAWAAAAZGlwL21vZGVscy9fX2luaXRfXy5wea2RTWvEIBCG7/6KuSUB2UO/DgFP7d7KHkq39BZsMmElUYMf3ULpf6+aSNK9tdTDKO+M76MzpDdaws4rdCDkpI2D4wEdmWVUbYdtTuxVqzs0D5i2pcSg3V5+Qru532r1rvDDNaNwmEvug3jAV/cYNEI67EHyARsZTMdScYmsiI4FBaGa9sSuryho79KRQoeTO7FbCj1y5wOd3d1QsIOY2LPxWNUEwko2MexGfUZTVkkV/ZxYADUYDBYq/bhMsEzKnBQpvHGLkZ+ZFHwQItSyGC/t576tgJ+d+xXq0nrp9+q9dPxf35/ntlK2Q/vD+w0XFuGFjx73xmhT9sVRDUqfFaS51/AZ0V9FRb4BUEsDBBQAAAAIAIhQaFsGu72aQAIAALMFAAASAAAAZGlwL21vZGVscy91bmV0LnB5lVTBjpswEL3nK9zT2lmCAtn2sCtfWimntIeqe4oi5JhhY8UxyDZbul/fcSAQknS19QWD37x582bMRB2q0nriSyt3sTFEOGJM1L/HRW2kV6UROhwtu5PJJIeCyNK8LpoFVSaTu4iUtccneyQWfG0N8sTfEJHmY0BEFhGpRJ4r88ITNplILZwjAfpVl3JPMe57mdca2OOE4AqpskwZ5bOMOtBFRC4yHmFhuboCS1ncw9lwhIFxUJzwse6O5GlApD2iU9xBxlxCeo5SVyD2f37C6pnO4zQoq7SQwH/ZGlgvvyjtb2HzVn1zJriz6kRIBw30+ltCG4br5NjzD/AfNosv0pNfHP3PofI7/jkiW+EgfPvyEJEat26vKteqf8/Wzq0hot/1QXLn+Lqjn9J0OlUs+EAUCiJWmBegRxVsM/YVjOR9VSvlfMgma8uPdfTYQCUDFeYZlJ6zxKKqwOR0mCxkiSTr6OQ4bQ5XaUe5WtnwCtZBTs/0zxLGbghAvhsCdm6t7pPNfbvbEFUMrhPQDsicRaQ9ZBcDh90j/OxWBdR8M1yr5B/z9nYmrwDhsS2bJ9Lwt3F90VbvQ41g6gNY4YGejLwor+GIxFF8atlOVTZshMLK1CeuwQw0s+QRg5exMh5sVeqQo4mIk0JDVuCol5bPYxzKQ5kDv9sqrQwIexcRodULdr+0Bu3nS4FGsQ+pxy5cqX9fQBrP/1dAV+74RoRS27+oFJ6uMc3RrfWMqvuUbbBvuTrwhN209ub/Afscjv4CUEsDBBQAAAAIAIhQaFsFozDzEAIAACYFAAAUAAAAZGlwL21vZGVscy9lbmNkZWMucHmNVMmS2yAQvesryGlgolF5SXKQi0sWn5xLltOUS4WhZVPGSANooszXp/GCLU9NyhyELbrfe/26UaZ3beMCCY2Tm8JaIjyxNk//i7qzMujGChOP5lmmoCaysc/Tfkq1reQmJ00X9rsPTivgY1YSB6FzFqGKLxg7Udeh0xR92HLSCqW0XWN2lkkjvCefTSO3FCG+N6ozwMqM4Ir8VaWtDlVFPZg6JwPsY1hcvmvBUVakcHY+wsTCwxNH+J/w1IENWhg6LOwImMcyFiC2f3/A4jcdFZNI2Rohgf9yHbA8wb6xTrDH6m/FZangunF/hFOHevuzv6cyaJ9c+2Zlo8B9hf12s318Ojk5yLE5Ctqw4R9zshIe4rtPH24zVm48fzwm3dPJ/b1mUT3RSEOcsGuge2y2HPYCrORJ60L7QNmMyM7xvboUG6FkhEKeMmUWom3BKnqYGMwqOT5SPZIdseSQU8ErzgHRQTM8g/Og6IX4hzFjZcK4Yt/4R/1+vMzJ/teSXQ0dauLnaxFjRsvzvRi/0fOXC/drEAFNXs5Iz1+GgvOV2UbRYLsdOBGAniy6yI+r5xiJUzM7oJ1K6NkgStdEv+MG7BnmYVxi8rzQNoBrGxM5erzMUhioaiHxw8FHBQ7ODseP36200RaEu8uJMHqNvWycRT/5XBgP7Cb1aPEr9f8XMClGNwqYJScSweXVwrbEo39QSwMEFAAAAAgAiFBoWygv4AM9AgAA5wUAABUAAABkaXAvbW9kZWxzL3Jlc3VuZXQucHmlVMGOmzAQvecrfLSzXpTAtodEvrRST2kP291ThCKvmWysEENt06X79R0DIZCkbaT6AszY7z2/mWGiD2VhPfGFVbvIGCIdMYb339G2Msrrwsg8pL50mUkGW6IK8zOpE6rNRu14UXl8sAWx4CtrECX6jBvibJTnCS9llmnzKuZsMlG5dI48gvuUF2pP8czXIqtyYIsJwRVYNhtttN9sqIN8ywNDkwrLVSVYyqJ+C1uSsCtSc3EUh5R45hiPr8elQOYVyP2vR1g901kUc6JNmUsF4slWwHo128K+SZu1YuqBFivqJalFC0c7FbRmrI+qGD+XR3u6jfWdHdrw/A38DS40jook7kwVCc+g9Dvxgb9IByHy8YFX+Ob2unTtFW6w7XSif+sPqZ0T6w59SuPpVLNgBtFoFLHSvAJtNLD0xBMwnYdDcPc7/KjAeC1zOu6bgLyepYyTixqMSsDGuFnxZpzorVpp5+n61HMBVKctuL6b41vCifNWZyCwuKcevHqJ+zlLz/jAqJfQoxecffOqFksFLKQ9B6jKP8l9QmZXFg6Ouhu9xxs8/J/uDP6puyHCmhK9JX3hCeQOyJzdTIStKMb+z1B9N/UIdH2E3gd9uQXpscnSfmJC79B37M4mE8myBJPhEJ1OBGk8tELQB6Y6gJUe6KlDBvBh1SJEmznsOPq6NrY3mb+zVeUVLizuBVNVNmijKDo8nrQFbmz/tUpiUWresq/vqb6LWYoOZvog5uwMe1xaLN9Q5/APg/6H1G9QSwMEFAAAAAgAiFBoW2iiX2lNAgAAEwYAABsAAABkaXAvbW9kZWxzL2NvbnZuZXh0X2xpdGUucHmdVMGK2zAQvecr1JuUKGbtLC1k0aVL20u69NDCQghGKymxWkcyktyYfH0lx5Ydd8OWmkBsz7w3b2aePJPHShsHnDasSJQC1AKlcHxO9rViTmpFyxD63EVmrKTWgsenj6Vmv6DP+6p5XQq0ngF/cbEHeS6VdHkOrSj3GLACA9FUVFlPRrLkrksNl60rYSBKIgQ9gIBK+Il46ketfmccegb/+4AryrlUB7LCB6PryhJWoEhVSM6FIlI5nz+PBXvC6pReM17ycdon0BD+8mnzAw6QbATp8j0yHYq2eQd6PLbob9TQo3C+o8sQz8JoC9MW41EoTmivzYkafhlQM5qHIc0DaEg3Atig+OTFwItO2Pfjw8gnGOFqo4ABi5GcedMvyqt/Es9uI534h21JlbOCrDKsa9feYS4qV5B7/EKtCG/e37+xP1ZYsu2y5zCbzyUKDQMJpFdJ1UHAlhPtrsdonThaEiVupHVxFS/BbDYXik0TIgerDWnVxzehKAtFvaL1qEZCq0ooDkd2qA1meBUNlqK/C/eo3vks5ISi7LqNyuifOdcnNVG6HbvPbuUunAz/v0j9XYaBdUZyQbLXh7VM0W5iu04aF9OZbKPEts4NwgwvU/wKbau/rm6o/+4pbKWt6Nto5fcN/U8b3mjkejR3nuliPwzSG4fmPDJh4/22CwflfLX6Ue1SKDjsH42wLZ4MMd/G+NQN64+BaJ8GXbHIPZDvyKTSMl3HA9z74sL0ptRhvTf0dosaFDeLxm6XUC4ytJt24GkmdbvPRr+DEPoDUEsDBBQAAAAIAIhQaFuWJwNloQAAANgAAAAdAAAAZGlwL29wZXJhdG9ycy9kb3duc2FtcGxlcnMucHktjE0KgzAQhfeeYnYqpJZ2KWQr3sBFKWWqkzoQJ2ESqfT0Lda3en98BS8xaIYcdJwbkcatMmYOgh4wQVdM5GBi50hJMuPT02MKb0m4RE/VZiCN6KluC/iJ3T9ae2lBKa8qsO1LbwawsDVpxki307W97/Xx6RqWTBqDx/yH8ods1Z/PO87AcLjawBImsuWTPQuhlgbQ80seY1AhTbZDn6guvlBLAwQUAAAACACIUGhbsLVxptIAAAAnAQAAFgAAAGRpcC9vcGVyYXRvcnMvbWFza3MucHk1kE1rAkEMhu/zK3LbGdgGtIfCwhZ68CCIiO1NRIJmdanzQXYsbH9958POIUzC+7z5UIN4C7v1BkYbvERYW7qyeibuYcMMNIELLUQv55u68AB3T5eTpelbB4q3Fqbxl/utd2w6BelZ6KsP+sCuiAyevfthibrZNKaoxqGAMCZ7HyHzFf+3sCicFTqHQ9e9LI4tpBLZcOe+NtiuPvarz6/qSCIJ0y5g+tGsrXlfLN8M0hTnwLoZ0uDxdfnsLxwf4upamM9wKuvqxJpDnqYtARGP6g9QSwMEFAAAAAgAiFBoW33yIO7CAAAAbAEAABMAAABkaXAvdGFza3MvY29tbW9uLnB5VZA7D4MwDIR3foW3hKpi6oTE1g5d+1qrqJgqCnGQE8RQ9b+X8CiQIdKdPlt3Tip2FrISyWmPoG3jOMBxlDflTTICvm2QGf1MXKO+oF8hmhqlKczEeZQDUGIFVhl8hl5JUhb3sNuZLs0T6B8V0cpq1yHLdLB01buFmHKJHBhDy7ROtiXngAu6jij9S9VYmC57Y5BiUGJ/SDcrpgLLhlUFafvvPx+FmIZZxcM9VN3iidmxrMSdDLmOILbN4RPLfUWa/ABQSwMEFAAAAAgAiFBoW7G41QDJAAAAVAEAABQAAABkaXAvdGFza3MvZGVub2lzZS5weV2PwWoDMQxE7/sVc3McFn/AQm4lx556L+6unJrYcrC0hFL6763jZUurg0DiaWY0xHwrVaGlzu+O2YWVZ42FfYIXnIc5eRE8EZco9OLlOg34KfaZTmbpa/NYLRSwePWvqYgchFIY8TGivMmI4/F6txMq6VoZZ5eFOtYBuwtUanH+XV/09FyYbLduFWrJcG7VmMRl0hpnwfbKTbju4Ob4aZqbmRBS8V3e/WbdQriolA/WjjBNY6fb0JiLIoZHF3BRtEygJPTn+mv4BlBLAwQUAAAACACIUGhbk7JoMgQBAAD2AQAAFQAAAGRpcC90YXNrcy9zdXBlcnJlcy5weXVQvW6EMAzeeQpvwAll6oTEemOH9vYoB841usRBsdGpqvruJYCgPakeLNn6/uzChTEmAYmp/1BEyk7Ui4tkPBiGc2FTDKBUHDGZGcRqiA9iE0aPiWFjD85aTEjizNWjPiBF7w0zvE8z/Q35YvjeFjAXmYBdyXmfkMtlN6AFrR050bpi9LYB7o3H7qVuIc9qHZe+MwYjRvvIvFE+G4hXbuB0uj/q1SxXQpkSwVkFxhX+b+hqljjs6kWv3v0S5pOfzG7SvUbCX37b3yZxnlVASa7f3zUypedgX2UOVbZgfTSrvDpOW31q5QRDVc+Ryqyxo/OQMTcBZ5fOQFEgZwL0jH/Y38UPUEsDBBQAAAAIAIhQaFs5L14sPQEAAD8CAAAUAAAAZGlwL3Rhc2tzL2lucGFpbnQucHldUcFqwzAMvecrRC9JSmroNSzHFXYZO4xdxgheorSithwsZ10Z+/fFSUhHfRDWs/Se9JyQ7Z0PEJxvTopZdQM3gRxrA1rgUMwvSWO0CDxxr4nDq5ZzmcB4WFusUprhdIJa7KCuiSnUdSZougLsWF89O8a8hIioCYhh7Wh10LVxIkvLtQD3KQVst+dLPmvFQ92NAEggkpbgNQnCmzYDPnrvfLZ58e6LWpyUoXMelhGJj5t8ZbPVSrZilxOZsU9xSxYe4DpdSrBQjSB+95rbbLcfB1Ry0j2+7z8KiPluf+P1GAbPcFBWcF7qurXTQlubryt7jM7f7XsMi1ErWeedBaWGQEaUxeCpEVh+rRf296o/aVRMS+iM0zO9urk76+SKAtoszwtII8daHZNYcwzR6hgF2IXJZ0Azuvy/+zf5A1BLAwQUAAAACACIUGhbbl9m2/EAAAApAgAAGgAAAGRpcC9vcHRpbS9nZXRfb3B0aW1pemVyLnB5jZE9T8NADIb3/Apv10ghKmxEuoGKtktZiGCtjsZJT9yXfBciQPx3LqlapVWGePKH/D6v5URqZylAsHQ4JhXW0GDYWxeklj9ICydIaJ+BERo5E5XQLANF/B7vHjLoUDbHsK/wIL75Ml9moK1GE1odq8e4hT4g2S++EcpjWiQQw/BeLFe2i/rp0JJ17HLmm4oVQBhaMidH+eAkL7fPFycRrmgEOicj2jm5MTgurrjDWZPgpzi5Jc/TJO0dWTct+/pS9sMZN82DqY+68dOo3WqzLa9Bp1US0iO8C9XimsjSomZv5tPYzsDl/QX89r/6Y2nyD1BLAwQUAAAACACIUGhb2iju87YAAAAvAQAAEQAAAGRpcC91dGlscy9zZWVkLnB5dY09C8IwEIb3/orgZKFkECelg6DSSQW7iEg5k9MGm0u5pIX+e/uxFMTb7n2e9y4ytnYchPOJYCDtbCKosXUnwAuqExEcqzLS+BIeURfYInehNPReDnu6XsWbSPQzleUQjiTe9nX5k46u8xKpNezovrjc8ux8ynbX7Ho47BeP1AeeqeN7aYEaqIr58QmoRsOcFlBVP+0nqA+S9oNNJDUGZGvI+GBUmnOD/90nkiot8Cc9QuUx+gJQSwMEFAAAAAgAiFBoWxifz349AQAAXAIAABUAAABkaXAvdXRpbHMvaW1hZ2VfaW8ucHmVkT9rwzAQxXd/itssteLqpARKi5cOLYFQStcQgrDl2BD9QZZD/O17lpwmQ5Z6MNb53nt3P2WNtxq+1xvotLM+wFrLg8rmgxm0G0H2YJyAYH3VZlmtGjhaWe+7qZM5GVoBBy/H8kMee8VfM6Cn0wcokxlap0zs41hZc1I+sHyTQ9dEGShSQf7z+Z7zKJXek9Q4pA85MnLiKPswOsWo2FB2eF7yp+VqhUXKSkYp+OpA7+2XNUog4i7+m5LudGHw0vTO9ootRSEWaQyvwuBN2honSvtIg5GA3/hGHr08qZnHWUBcNeUESjhjrYKsWkbrH6V2bIrAyg1USJZ8W+wumwTsW+kUVcpycR220+TEApUfaPHHAld/UPKhM+El528z7mnWCzoBRJrjNF+6gTscZusbCgtBHPh/g+IN3kb9AlBLAwQUAAAACACIUGhb2TxThjgCAADwBAAAFAAAAGRpcC91dGlscy9tZXRyaWNzLnB5hVPbbtwgEH3fr+ARfKHGUdXGqfuyUhQpH5BIUWTRNb6ovi2wienXdzDY2kulWjLGwzmHw8ywa/tplBrpUR6ayH3oMNDqNBx0Ow68Q1yhxwj1XDe7XSkqNKlB4jlCxgbn4oN3OaNJhMSkcibi7yTbIXh6JVCOHil8i25UCs/00PF+wknECLDP/shCaCvLoa0WPSZ5nmRICn2SgzelxaBGie/v7V6l+GgPIp+pmzgBD08TmgSO0401S/AF31u+lSAoRuyaCoZCOBdxJ1eq7Qt4p07cJsAf+5lFz2kOOiyC4W6J7VmOn1ngwSQI0ge0TyGWnsdc1k7FnD9S/lEX0zh2aQkbfYtYdEce7Jq5XDN+baGqtu75DTtY+bGVDuxwhr7WC8wZ2li0Odc21+LmWtzDbaJ6PuUYp8G2FO4ZCSCwioX7lJAvGG/Gwm1TB/XA0Htd8OeVXrehveAD9kXSo+Yd5FS23DZw0TE8+9qUDXSkKy//BQ35RimNWBZl77GbZzGDH7LqLZzPf3AyYG0cy7rkeHtlE5afzlTT1k1RSXEsxCBkbTZLr5t6VWkqYRhsa5Vtn+M4jeL1cvS8vvDx6sJP0QuEZ6oaPom3OM3el7AxF7Lw2r3xEwgvrUr1iLfGp6dBHU9C/BGYOdV5vnG18F/+x0/8+eXGV0epsTHQ3uE8w7ieRv0GCAbgT5TQ9GsgJYWrgAmhVTdy7RP52bSdWNB0gJSgHzYPyzRbNZbFWweuAnDb68AitvL8BVBLAwQUAAAACACIUGhbD8k0JAkBAACVAgAAFgAAAGRpcC91dGlscy9lYXJseXN0b3AucHmFUstuwyAQvPsr9pZEcSzaY11y67WXfgAi8VpFwoBgHSV/X+MH2GmlclgBy8zOjChU56wn6CR9F1ctQ4AP6fXji6xz6N8KGFaDLQihjCIh9gF1W0JnG+S7Tt53JThJCs0V+StjQ0cZ0aAmyVnFDhNBXBFXjbBY6umcoMtmvs8kabclumAgfoqqK2VaUO2kaJIEqAPC0twCTd+Ji2w4qzORCCQJ+ac1uL5VhJ6zlEAgdLP7m9Q9lpBhJcTHgwdr+Nr0kK63N2yAw34EwTkPgOOT2UP0kXNam5nR7yv06Rmdp7ZpcJayjW6kq/+NZKy/MkleE3nU+MeomfnIX1LPI/XebNpLJssfKH4AUEsDBBQAAAAIAIhQaFuSBdKZgwAAAPwAAAAYAAAAY29uZmlncy9ncmlkX3NlYXJjaC5qc29uTY7BCsIwEETv/YqQc5GtWg/+ingIyaoBuynptojiv5sQkvU2s/OY2U+nlCZkfVaXJJNZs+uLRrIObXURl//QBtoIX6yTveabdjjzozUdCje2+IaG11QixL4gp2Njwsx+8m+M8tByd3XSODPJ3FMg2AGMfZVDQzxjNOwDyegAADnvvj9QSwECFAMUAAAACACIUGhbH3aCXiQAAAAjAAAAEAAAAAAAAAAAAAAApIEAAAAAcmVxdWlyZW1lbnRzLnR4dFBLAQIUAxQAAAAIAIhQaFuvISAQKgAAACgAAAAJAAAAAAAAAAAAAACkgVIAAABSRUFETUUubWRQSwECFAMUAAAACACIUGhbg8INAegFAAApEQAACAAAAAAAAAAAAAAApIGjAAAAdHJhaW4ucHlQSwECFAMUAAAACACIUGhbqrTgd8oCAAAmBgAAEwAAAAAAAAAAAAAApIGxBgAAZXhwZXJpbWVudF9zd2VlcC5weVBLAQIUAxQAAAAIAIhQaFstw3Or6QMAAAQKAAASAAAAAAAAAAAAAACkgawJAABzYW1wbGVfZW5zZW1ibGUucHlQSwECFAMUAAAACACIUGhbkhtfyakFAACODwAAEgAAAAAAAAAAAAAApIHFDQAAYW5hbHl6ZV9scl9iaWFzLnB5UEsBAhQDFAAAAAgAiFBoW+N/AHAKAQAAkgIAABYAAAAAAAAAAAAAAKSBnhMAAGRpcC9tb2RlbHMvX19pbml0X18ucHlQSwECFAMUAAAACACIUGhbBru9mkACAACzBQAAEgAAAAAAAAAAAAAApIHcFAAAZGlwL21vZGVscy91bmV0LnB5UEsBAhQDFAAAAAgAiFBoWwWjMPMQAgAAJgUAABQAAAAAAAAAAAAAAKSBTBcAAGRpcC9tb2RlbHMvZW5jZGVjLnB5UEsBAhQDFAAAAAgAiFBoWygv4AM9AgAA5wUAABUAAAAAAAAAAAAAAKSBjhkAAGRpcC9tb2RlbHMvcmVzdW5ldC5weVBLAQIUAxQAAAAIAIhQaFtool9pTQIAABMGAAAbAAAAAAAAAAAAAACkgf4bAABkaXAvbW9kZWxzL2NvbnZuZXh0X2xpdGUucHlQSwECFAMUAAAACACIUGhblicDZaEAAADYAAAAHQAAAAAAAAAAAAAApIGEHgAAZGlwL29wZXJhdG9ycy9kb3duc2FtcGxlcnMucHlQSwECFAMUAAAACACIUGhbsLVxptIAAAAnAQAAFgAAAAAAAAAAAAAApIFgHwAAZGlwL29wZXJhdG9ycy9tYXNrcy5weVBLAQIUAxQAAAAIAIhQaFt98iDuwgAAAGwBAAATAAAAAAAAAAAAAACkgWYgAABkaXAvdGFza3MvY29tbW9uLnB5UEsBAhQDFAAAAAgAiFBoW7G41QDJAAAAVAEAABQAAAAAAAAAAAAAAKSBWSEAAGRpcC90YXNrcy9kZW5vaXNlLnB5UEsBAhQDFAAAAAgAiFBoW5OyaDIEAQAA9gEAABUAAAAAAAAAAAAAAKSBVCIAAGRpcC90YXNrcy9zdXBlcnJlcy5weVBLAQIUAxQAAAAIAIhQaFs5L14sPQEAAD8CAAAUAAAAAAAAAAAAAACkgYsjAABkaXAvdGFza3MvaW5wYWludC5weVBLAQIUAxQAAAAIAIhQaFtuX2bb8QAAACkCAAAaAAAAAAAAAAAAAACkgfokAABkaXAvb3B0aW0vZ2V0X29wdGltaXplci5weVBLAQIUAxQAAAAIAIhQaFvaKO7ztgAAAC8BAAARAAAAAAAAAAAAAACkgSMmAABkaXAvdXRpbHMvc2VlZC5weVBLAQIUAxQAAAAIAIhQaFsYn89+PQEAAFwCAAAVAAAAAAAAAAAAAACkgQgnAABkaXAvdXRpbHMvaW1hZ2VfaW8ucHlQSwECFAMUAAAACACIUGhb2TxThjgCAADwBAAAFAAAAAAAAAAAAAAApIF4KAAAZGlwL3V0aWxzL21ldHJpY3MucHlQSwECFAMUAAAACACIUGhbD8k0JAkBAACVAgAAFgAAAAAAAAAAAAAApIHiKgAAZGlwL3V0aWxzL2Vhcmx5c3RvcC5weVBLAQIUAxQAAAAIAIhQaFuSBdKZgwAAAPwAAAAYAAAAAAAAAAAAAACkgR8sAABjb25maWdzL2dyaWRfc2VhcmNoLmpzb25QSwUGAAAAABcAFwDrBQAA2CwAAAAA""")
open("/content/dip_research/dip_code.zip","wb").write(data)
with zipfile.ZipFile(io.BytesIO(data),'r') as z:
    z.extractall("/content/dip_research")
os.chdir("/content/dip_research")
print("CWD:", os.getcwd())
print("Some files:")
print("\n".join(sorted(glob.glob("**/*", recursive=True))[:20] + ["..."]))


## 3) Prepare demo images

In [ ]:

import os, io, numpy as np
from PIL import Image, ImageDraw
import requests

os.makedirs("demo", exist_ok=True)
url = "https://raw.githubusercontent.com/scikit-image/scikit-image/main/skimage/data/astronaut.png"
try:
    data = requests.get(url, timeout=20).content
    Image.open(io.BytesIO(data)).convert("RGB").save("demo/clean.png")
    print("Downloaded demo/clean.png")
except Exception as e:
    print("Fallback synthetic RGB:", e)
    H,W=256,256
    img = Image.fromarray(np.tile(np.linspace(0,255,W,dtype=np.uint8),(H,1))).convert("RGB")
    img.save("demo/clean.png")

# Noisy
clean = Image.open("demo/clean.png").convert("RGB")
arr = np.array(clean).astype(np.float32)/255.0
rng = np.random.default_rng(0)
noisy = np.clip(arr + rng.normal(0, 25/255.0, size=arr.shape), 0, 1)
Image.fromarray((noisy*255+0.5).astype(np.uint8)).save("demo/noisy.png")

# SR×4 LR
clean.resize((clean.width//4, clean.height//4), resample=Image.BICUBIC).save("demo/lr_x4.png")

# Inpainting mask + occluded
mask = Image.new("L", clean.size, 255)
d = ImageDraw.Draw(mask)
d.rectangle([int(clean.width*0.3), int(clean.height*0.3), int(clean.width*0.7), int(clean.height*0.7)], fill=0)
mask.save("demo/mask_center.png")
obs = np.array(clean); m = (np.array(mask)>0).astype(np.uint8); obs[m==0]=0
Image.fromarray(obs).save("demo/occluded.png")

!ls -la demo


## 4) Denoising demo

In [ ]:

%%bash
python train.py --task denoise --image demo/noisy.png --gt demo/clean.png   --net unet --depth 5 --features 64 --skip   --optimizer adam --lr 0.01 --iterations 1200 --early_stop 200   --save_dir runs/denoise_demo


In [ ]:

from PIL import Image
import matplotlib.pyplot as plt
fig,ax = plt.subplots(1,3, figsize=(12,4))
ax[0].imshow(Image.open("demo/noisy.png")); ax[0].set_title("Noisy"); ax[0].axis("off")
ax[1].imshow(Image.open("runs/denoise_demo/final.png")); ax[1].set_title("DIP final"); ax[1].axis("off")
ax[2].imshow(Image.open("demo/clean.png")); ax[2].set_title("GT"); ax[2].axis("off")
plt.show()


## 5) Super‑résolution ×4 demo

In [ ]:

%%bash
python train.py --task superres --image demo/lr_x4.png --scale 4   --net resunet --depth 5 --features 64 --optimizer adam --lr 0.01   --iterations 1000 --save_dir runs/sr_x4_demo


In [ ]:

from PIL import Image
import matplotlib.pyplot as plt
fig,ax = plt.subplots(1,3, figsize=(12,4))
ax[0].imshow(Image.open("demo/lr_x4.png").resize(Image.open("demo/clean.png").size, Image.BICUBIC)); ax[0].set_title("LR→bicubic"); ax[0].axis("off")
ax[1].imshow(Image.open("runs/sr_x4_demo/final.png")); ax[1].set_title("DIP SR×4"); ax[1].axis("off")
ax[2].imshow(Image.open("demo/clean.png")); ax[2].set_title("GT"); ax[2].axis("off")
plt.show()


## 6) Inpainting demo (mask central)

In [ ]:

%%bash
python train.py --task inpaint --image demo/occluded.png --mask demo/mask_center.png   --net encdec --depth 6 --features 64 --optimizer adam --lr 0.01   --iterations 1500 --save_dir runs/inpaint_demo


In [ ]:

from PIL import Image
import matplotlib.pyplot as plt
fig,ax = plt.subplots(1,3, figsize=(12,4))
ax[0].imshow(Image.open("demo/occluded.png")); ax[0].set_title("Occluded"); ax[0].axis("off")
ax[1].imshow(Image.open("runs/inpaint_demo/final.png")); ax[1].set_title("DIP inpaint"); ax[1].axis("off")
ax[2].imshow(Image.open("demo/clean.png")); ax[2].set_title("Ref."); ax[2].axis("off")
plt.show()


## 7) Mini‑grille : architecture × optimiseur × LR (dénoyautage)

In [ ]:

%%bash
cat > small_grid.json <<'JSON'
{
  "net": ["unet", "encdec"],
  "depth": [4],
  "features": [64],
  "optimizer": ["adam", "sgd"],
  "lr": [0.005, 0.01],
  "iterations": [600]
}
JSON
python experiment_sweep.py --task denoise --image demo/noisy.png --gt demo/clean.png   --grid small_grid.json --save_dir runs/sweep_demo


## 8) Échantillonnage / Ensemblage (multi‑seeds)

In [ ]:

%%bash
python sample_ensemble.py --task denoise --image demo/noisy.png   --net unet --depth 5 --features 64 --optimizer adam --lr 0.01   --iterations 800 --seeds 4 --aggregate mean --save_dir runs/denoise_ens


## 9) LR → lissage (SGD)

In [ ]:

%%bash
python analyze_lr_bias.py --task denoise --image demo/noisy.png --gt demo/clean.png   --net unet --depth 5 --features 64 --optimizer sgd   --lrs 0.0005 0.001 0.005 0.01 0.05 --iterations 600   --save_dir runs/lr_bias_demo


In [ ]:

from PIL import Image, ImageOps
from IPython.display import display
for p in ["tv_vs_lr.png","hf_vs_lr.png","psnr_vs_lr.png"]:
    display(Image.open(f"runs/lr_bias_demo/{p}"))


## 10) (Optionnel) Sauvegarder les sorties dans Google Drive

In [ ]:

# from google.colab import drive
# drive.mount('/content/drive')
# !cp -r runs /content/drive/MyDrive/DIP_runs
print("Uncomment to save outputs to Drive.")
